# Detecting False Arrythmias in the ICU
### - A Physiological Signal Processing Challenge -
##### Los 0011 Fantásticos: Emilio Pareja Flores, Amanda Román Román y Sara Valiente Jaén

Throughout this code, we aim to come up with a solution to detect arrythmias in ICUs in a more efficient way. Many times, a monitor raises an arrythmia alarm even though the patient being monitored is not having an arrythmia. This causes additional stress to both patients and medical staff in ICUs, hospital stay durations might be increased and the staff might be being worn off. Therefore, it is of great importance to improve detection algorithms.

We have divided this notebook into sections so navigation is easier and the code is better understood.

***
### Importing neccesary libraries and modules
***

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

import numpy as np # linear algebra
from numpy.fft import *
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

%matplotlib inline
from scipy.io import loadmat
import matplotlib.pyplot as plt

from IPython.display import display, clear_output

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

!pip install biosppy
from biosppy.signals.ecg import hamilton_segmenter
from biosppy.signals.ecg import ecg

import scipy
from scipy import signal
from scipy import integrate
from scipy.signal.windows import *

***
### Initial steps
***
<div class="alert alert-block alert-info">
Defined functions
</div>


In [ ]:
def plot_signal(path, file):
    '''
    This function takes as arguments a path and a file type ".hea". 
    
    The ".mat" file associated to the passed file is found and some information is
    extracted, such as the sampling frequency and the signal\' labels.
    
    A loop is implemented in order to go over every label, every signal of the file, and
    plot it using Matplotlib.
    '''
    
    pat = loadmat(os.path.join(path,file[:-4]+'.mat'))
    
    with open(os.path.join(path,file)) as f:
            lines = f.readlines()

    fs = int(lines[0].split()[2]) #hz
    num_signals = int(lines[0].split()[1])

    labels = []
    for signal in lines[1:]:
        #split the string and get the last which is the label
        labels.append(signal.split()[-1])    
    print(labels)
    ind=len(labels)
    #int(len(labels)/2)

    signals = pat['val']

    t = np.arange(len(signals[0]))/fs
    
    plt.figure(figsize=[12,10])
    
    for l in range(len(labels)):
        plt.subplot(int('{}{}{}'.format(2,2,l+1)))
        plt.plot(t,signals[l,:])
        plt.title('{}'.format(labels[l]))
        
        
    return None
        



def periodogram(x,Nfft):
    
    N = len(x)
    x_f = fft(x,Nfft) 
    Px = (np.abs(x_f)**2) / N
    Px = fftshift(Px)
    f = fftshift(fftfreq(Nfft))
    return Px, f # periodogram of the signal and the frequencies it takes


<div class="alert alert-block alert-info">
Displaying different kinds of signals
</div>

Let's first plot different signals in order to assess visually what their differences might be.

In [ ]:
# True alarm, id 1

path = '/kaggle/input/physiological-signals-processing-challenge-2021/data_challenge/training/'

plot_signal(path,'1.hea')

In [ ]:
# False alarm, id 4

plot_signal(path,'4.hea')

In [ ]:
#NaN mean heart rate, id 213

plot_signal(path,'213.hea')

In [ ]:
#NaN mean heart rate, id 363

plot_signal(path,'363.hea')


<div class="alert alert-block alert-info">
Initial filtering tests
</div>
Now, we are going to try to reduce noise and trends of a single signal applying filtering.

First, we just plot an original signal. In this case, we take '101.hea'.

In [ ]:
pat_filename = '101.hea'
pat = loadmat(os.path.join(path,pat_filename[:-4]+'.mat'))

#get ECG

with open(os.path.join(path,pat_filename)) as f:
        lines = f.readlines()

#get fs
fs = int(lines[0].split()[2]) #hz

#number of signals
num_signals = int(lines[0].split()[1])

#get labels
labels = []
for signal in lines[1:]:
    #split the string and get the last which is the label
    labels.append(signal.split()[-1])
    
print(labels)

#get signals
signals = pat['val']

#get one ECG signal
l = len(signals[0])-10000

t = np.arange(10000)/fs

#plt.plot(t,signals[0,:1000])

plt.figure(figsize=[15,8])

plt.subplot(221)
plt.plot(t,signals[0,l:]) # II signal
plt.title('II signal')

plt.subplot(222)
plt.plot(t,signals[1,l:]) # V signal
plt.title('V signal')

plt.subplot(223)
plt.plot(t,signals[2,l:]) # PLETH signal
plt.title('PLETH signal')

plt.subplot(224)
plt.plot(t,signals[3,l:]) # ABP signal
plt.title('ABP signal')

Next, we **detrend** the signal and apply a **bandpass** filter (between 0.5 and 40 Hz). The result is our **filtered** signal.

In [ ]:
sig = signals[0,:]
sig_detrended = scipy.signal.detrend(sig)
t = np.arange(len(sig))/fs


f_low = 0.5        # Low cutoff frequency
f_high = 40        # High cutoff frequency
numtaps = 64       # Filter length
b_bp = scipy.signal.firwin(numtaps, [f_low, f_high], pass_zero=False, fs=fs)     # Create FIR bandpass filter (0 DC gain)
sig_f = scipy.signal.filtfilt(b_bp, 1, sig_detrended)    # Apply bandpass filter (detrended signals), denominator 1


plt.figure(figsize=[10,5])

plt.subplot(2,2,1)
plt.plot(t,sig)
plt.plot(t,sig_detrended)
plt.xlim([100,105])
plt.legend(('Original II lead signal', 'Detrended'))

plt.subplot(2,2,2)
plt.plot(t,sig)
plt.plot(t,sig_f)
plt.xlim([100,105])
plt.legend(('Original II lead signal', 'Detrended+BP'))

Plotting the **periodogram** of the original and the filtered signals yields the following:

In [ ]:
L = len(sig)  # Length of the signal
n_L = np.arange(0, L)
nfft = 1024 # number of points of the FT

P_sig, f_sig = periodogram (sig, nfft)     # Compute the periodogram (original signal)
P_sig_f, f_sig_f = periodogram (sig_f, nfft)     # Compute the periodogram (filtered signal)

plt.figure(figsize=[10,10])

plt.subplot(311)
plt.plot(n_L, sig)
plt.plot(n_L, sig_f)
plt.xlabel('$n$')
plt.ylabel('Signal')
plt.legend(('Original', 'Filtered'))

plt.subplot(312)
plt.plot(f_sig, 10*np.log10(P_sig))
plt.plot(f_sig_f, 10*np.log10(P_sig_f))
plt.title('Periodogram of the signal')
plt.xlabel('Normalized frequency')
plt.ylabel('Magnitude (dB)')
plt.legend(('Original', 'Filtered'))

plt.subplot(313)
plt.plot(f_sig, 10*np.log10(P_sig))
plt.plot(f_sig_f, 10*np.log10(P_sig_f))
plt.xlim(0, f_sig[-1])
plt.title('Periodogram of the signal (only positive frequencies)')
plt.xlabel('Normalized frequency')
plt.ylabel('Magnitude (dB)')
plt.legend(('Original', 'Filtered'))

plt.tight_layout()

Let us now compare the obtained regular periodogram with the modified periodogram and the Welch's periodogram (all of the filtered signal).

In [ ]:
PSD = []
freqs = []

sig_div = np.split(sig_f,100)

L = len(sig_div[0])

window = scipy.signal.windows.hamming(L,sym=False)

for a in sig_div:
    a_w = a * window
    Px_window,f_window = periodogram(a_w,nfft)
    PSD.append(Px_window)
    freqs.append(f_window)

Px_welch,f_welch = scipy.signal.welch(sig_f,fs=fs) 

plt.figure(figsize=[10,10])

plt.subplot(311)
plt.plot(f_sig_f, 10*np.log10(P_sig_f))
plt.title('Periodogram')
plt.xlabel('Normalized frequency')
plt.ylabel('Magnitude (dB)')

plt.subplot(312)
#plt.plot(f_window, 10*np.log10(PSD).T)
plt.plot(f_window, 10*np.log10(np.mean(PSD,axis=0)))
plt.title('Modified periodogram (Hamming window)')
plt.xlabel('Normalized frequency')
plt.ylabel('Magnitude (dB)')


plt.subplot(313)
plt.plot(f_welch, 10*np.log10(Px_welch))
plt.title('Welch\'s periodogram')
plt.xlabel('Normalized frequency')
plt.ylabel('Magnitude (dB)')


plt.tight_layout()


It is clearly seen that the modified periodogram is the one that has less noise associated. Therefore, in our code we will use the **modified periodogram (with a Hamming window)** to compute the power spectrum of the input sets signals.

In order to do so, we partition the signal into 100 segments and compute their modified periodogram applying a Hamming window. The corresponding power is added to the array PSD, which we plot as the power spectrum of the signal. The total power is the sum of the all the powers (numbers) contained in the PSD.

***
### Working with the training data set
***

After having obtained a general idea of the signals we are working with and the filtering they need, we are ready to enter the input data set and start defining the parameters we are going to be looking at.

<div class="alert alert-block alert-info">
Defined functions
</div>

These functions are, roughly, the **core of our project**. They plot and filter the signals, get information, compute parameters, get rid of useless and/or problematic values, and build the parameters array.

The **parameters** which will be passed to the decisor in order to discriminate
between true and false alarm are:
    
    - ECG: Mean heart rate
    - ECG: Standard deviation of the heart rate
    - ECG: Activity (Hjorth 0)
    - ECG: Mobility (Hjorth 1)
    - ECG: Complexity (Hjorth 2)
    - ECG: Shannon entropy
    - ABP: Total energy
    - PLETH: Total energy
    - RESP: Total energy

In [ ]:
def histogram (ids_alarms,ids_params,title):
    '''
    Takes an array [[ids] [corresponding alarm number; 1 being true alarm and 0, false]],
    another array [[ids] [parameter]] and a string called "title", which refers to the 
    parameter being studied (and passed to the function in ids_params as numbers).
    
    ids_alarms is converted into a dictionary to compare the ids (dictionary's keys) with
    the ids_params' ids, so that the ids corresponding to a true alarm will have their
    parameters stored in the list "true" and those corresponding to a false alarm, in 
    "false".
    
    Two histograms are plotted into a single figure: The one corresponding to true alarms
    and the other one, to false alarms. That way, true alarms and false alarms's populations
    are easily seen, and its overlapping can be visualized.
    
    Poor parameters will have the true and false populations almost completely overlapped.
    '''
    d = {}
    true = []
    false = []
    p = 0
    
    for i,a in zip(ids_alarms[0],ids_alarms[1]):
        d[i] = a
         
    for x in ids_params[0]:    
        if d[x] == 1:
            true.append(ids_params[1][p])
        else:
            false.append(ids_params[1][p])
        p += 1
            
    plt.hist(true, 100, alpha=0.5, label='true alarm')
    plt.hist(false, 100, alpha=0.5, label='false alarm')
    plt.title(title)
    plt.legend(loc='upper right')

    plt.show()
    
    return None


def filter_it(signals,fs):
    '''
    Filters the passed signal as explained before: detrend + bandpass filter (0.4-50 Hz);
    returns the filtered signal.
    '''
    
    #sig = signals[0,:] # II lead???
    sig=signals
    sig_detrended = scipy.signal.detrend(sig)
    t = np.arange(len(sig))/fs

    f_low = 0.5        # Low cutoff frequency
    f_high = 40        # High cutoff frequency
    numtaps = 64       # Filter length
    
    b_bp = scipy.signal.firwin(numtaps, [f_low, f_high], pass_zero=False, fs=fs)     # Create FIR bandpass filter (0 DC gain)
    sig_banddet = scipy.signal.filtfilt(b_bp, 1, sig_detrended) # bandpass and detrended
    
    return sig_banddet



def hj_activity (array):
    '''Computed Hjorth descriptor 0, which represents the total signal power.'''
    
    return np.var(array)



def hj_mobility (array):
    '''Computed Hjorth descriptor 1, which reflects the dominant frequency.'''
    
    return np.sqrt(np.divide(hj_activity(np.diff(array, axis=0)),hj_activity(array)))



def hj_complexity (array):
    '''Computed Hjorth descriptor 2, related to the bandwidth.'''
    
    return np.divide(hj_mobility(np.diff(array,axis=0)), hj_mobility(array))



def compute_entropy(data):
    '''Computed the Shannon entropy of the passed signal.'''
    
    prob1 = pd.value_counts(data) / len(data)
    
    return sum(np.log2(prob1) * prob1 * (-1))
 


def get_parameters(pat,header,plot_flag = True):
    """
    Gets the parameters which will be passed to the decisor in order to discriminate
    between true and false alarm. These descriptors are:
    
        - ECG: Mean heart rate
        - ECG: Standard deviation of the heart rate
        - ECG: Activity (Hjorth 0)
        - ECG: Mobility (Hjorth 1)
        - ECG: Complexity (Hjorth 2)
        - ECG: Shannon entropy
        - ABP: Total energy
        - PLETH: Total energy
        - RESP: Total energy
    
    If the file being studied does not have ABP and/or PLETH and/or RESP signals,
    the corresponding energy is set to zero.
    """

    #Read header and get labels and signals, and from that extract one ECG signal

    #ECG leads in the training set
    ecg_lead = ['I', 'II', 'III', 'MCL','V','aVF', 'aVL','aVR']


    #get fs
    fs = int(header[0].split()[2]) #hz

    #number of signals
    num_signals = int(header[0].split()[1])
    
    #get labels
    labels = []
    for signal in header[1:]:
    #split the string and get the last which is the label
        labels.append(signal.split()[-1])


    #get signals
    signals = pat['val']

   #find the ECG signals
    idx_ecg = [labels.index(i) for i in ecg_lead if i in labels]

        
    #handle not enough beats
    
    try:
        ts, filte,rpeaks, temp_ts,templates, hr_ts,hr = ecg(signal=signals[idx_ecg[0],:], sampling_rate=fs, show=False)
    except:
        print("Not enough beats")
        print(header[0].split()[0])
        hr = 60
        templates = 0
        
    mean_hr = np.mean(hr)
    std_hr = np.std(hr)
    
    filtered = filter_it(signals[idx_ecg[0],:],fs) # ecg lead signal, filtered
    
    PSD = []
    sig_div = np.split(filtered,100)

    L = len(sig_div[0])
    window = scipy.signal.windows.hamming(L,sym=False)

    for a in sig_div:
        a_w = a * window
        Px_window,f_window = periodogram(a_w,nfft)
        PSD.append(Px_window)
        

    total_power = np.sum(PSD)

    activity = hj_activity(filtered)
    mobility = hj_mobility(filtered)
    complexity = hj_complexity(filtered)
    
    #entropy = scipy.stats.entropy(filtered)
    entropy = compute_entropy(filtered)
    
    #plt.figure(figsize=[10,10])
    if plot_flag:
        
        plt.subplot(311)
        plt.plot(signals[idx_ecg[0],:])
        plt.title('ECG lead')

        plt.subplot(312)
        plt.plot(np.arange(0,len(filtered)),filtered)
        plt.title('ECG lead, filtered')
        
        plt.subplot(313)
        # plt.plot(f_window,10*np.log10(Px_window))
        plt.plot(f_window, 10*np.log10(np.mean(PSD,axis=0)))
        plt.title('Power spectrum')
        
        plt.tight_layout()
        

    #find energies
        
    en_a=0
    en_p=0
    en_r=0
        
    for l in labels:
        if l == 'ABP':
            abp_signal = signals[labels.index(l),:]
            en_a = sum(np.absolute(a)**2 for a in abp_signal)
        elif l == 'PLETH':
            pleth_signal = signals[labels.index(l),:]
            en_p = sum(np.absolute(a)**2 for a in pleth_signal)
        elif l == 'RESP':
            resp_signal = signals[labels.index(l),:]
            en_r = sum(np.absolute(a)**2 for a in resp_signal)
        
    
    return mean_hr,std_hr,activity,mobility,complexity,entropy,en_a,en_p,en_r


def clean_array(np_array):
    ''' 
    This function aims to tackle nan, inf and -inf values. 
    
    An array, containing problematic values, is passed; the same array is returned, but
    without nan, positive and negative infinite values.
    
    Since different kinds of arrythmias are not to be detected, there is no distinction
    between problematic values, they are all set to zero.
    '''
    
    array_list = [] 
    clean = []

    for i in range(len(np_array[0,:])):
        array_list.append(np_array[:,i])

    for array in array_list:
        array = np.array(array,dtype='f')
        clean.append(np.nan_to_num(array, copy=False, nan=0.0, posinf=0.0, neginf=0.0))

    clean = np.array(clean)
    clean = clean.T
    
    return clean



<div class="alert alert-block alert-info">
Reading data
</div>

Let us get into the training set and **compute the parameters** for each recording!

In [ ]:
#get all the possible labels in the training set

for dirname, _, filenames in os.walk('/kaggle/input/physiological-signals-processing-challenge-2021/data_challenge/training/'):
    filenames.sort()
    j = 0
    labels = []
    
    for filename in filenames:
        if 'mat' in filename:
            continue
        clear_output(wait = True)
        print(j," of ",len(filenames)/2)
        j+=1
        
        with open(os.path.join(dirname,filename)) as f:
                header = f.readlines()
    
        for signal in header[1:]:
            #split the string and get the last which is the label
            labels.append(signal.split()[-1])


print(np.unique(labels))

ecg_lead = ['I', 'II', 'III', 'MCL','V','aVF', 'aVL','aVR']

In [ ]:
#get into the data folder
from IPython.display import display, clear_output

X = []

for dirname, _, filenames in os.walk('/kaggle/input/physiological-signals-processing-challenge-2021/data_challenge/training/'):
    filenames.sort()
    j = 0
    for filename in filenames[:]:
        #print(filename)
        
        if j%5==0:
            clear_output(wait = True)
            print(j," of ",len(filenames)/2)
            #print(filename)
            
        if 'mat' in filename:
            continue

        #read the pat data
        j+=1
        record = loadmat(os.path.join(dirname,filename[:-4]+'.mat'))
        
        #get the header
        with open(os.path.join(dirname,filename)) as f:
            header = f.readlines()

        # get parameters
        mean_hr,std_hr,activity,mobility,complexity,entropy,en_a,en_p,en_r=get_parameters(record,header,plot_flag = True)
        
        X.append([int(filename[:-4]),mean_hr,std_hr,activity,mobility,complexity,entropy,en_a,en_p,en_r])



<div class="alert alert-block alert-info">
Formatting the parameters list
</div>

The parameters list X contains nan,inf and -inf values which would raise errors later on in the code if they were not hadled. Consequently, X is passed to our previously defined function that gets rid of these values. 

The resulting array Z does not have problematic values but still has all the information that X stores. **Z is the array that will be passed to our decisor!**

In [ ]:
# nan and +/-inf handling:

X = np.array(X)

Z = clean_array(X)


<div class="alert alert-block alert-info">
Final steps of the training prediction
</div>


The array containing the ids and their corresponding 1 (true alarm) and 0 (false alarm) is built, and the histograms of some parameters are plotted in order to assess their efficacy.

In [ ]:
#Get labels for the patients

y = np.loadtxt('/kaggle/input/physiological-signals-processing-challenge-2021/alarms_training.csv',skiprows=1,delimiter = ',',usecols = [0,1])

y_train = y[np.array(Z[:,0]-1,dtype = np.int32),:]

print(y_train.shape)
print(Z[:,1].shape)
print(y_train[:,1].shape)

histogram(y_train.T,[Z[:,0],Z[:,1]],'mean_hr')
histogram(y_train.T,[Z[:,0],Z[:,2]],'std_hr')
histogram(y_train.T,[Z[:,0],Z[:,3]],'activity')
histogram(y_train.T,[Z[:,0],Z[:,4]],'mobility')
histogram(y_train.T,[Z[:,0],Z[:,5]],'complexity')
histogram(y_train.T,[Z[:,0],Z[:,6]],'entropy')

Lastly, the **Naive Bayes decisor** is finally aseembled. It is trained through Z and y_train and then used to predict y_hat, predictions in terms of the training set. The accuracy is displayed.

In [ ]:
#Create naive bayes
from sklearn.naive_bayes import GaussianNB

#get priors

P_h0 = np.mean(y_train[:,1] == 0)
P_h1 = np.mean(y_train[:,1] == 1)

print('P_h0:',P_h0,'; P_h1:',P_h1)

#naive bayes model

nb_detector = GaussianNB(priors = [P_h0,P_h1])

#train the model

nb_detector.fit(Z[:,1:],y_train[:,1])

print("mean values n_classes, n_features")
print(nb_detector.theta_)

print("variance values n_classes, n_features")
print(nb_detector.sigma_)

Now that we have our parameters obtained we can apply the following rule

$$P(H_1|\boldsymbol{x}) \mathop{\gtrless}^{D_1}_{D_0}P(H_0|\boldsymbol{x})$$

In [ ]:
#predict first case
'''
x_1 = X[0,1:]

D = nb_detector.predict(x_1[np.newaxis,:])

print(X[0,0])
print(y_train[0,0])
print('Detection: %d'%D[0])
print('Hypothesis H: %d' %y_train[0,1])
'''

#whole training set
y_hat = nb_detector.predict(Z[:,1:])
print(Z[:,1:].shape)

print('ACC = %.2f'%np.mean(y_hat == y_train[:,1]))

***
### Working with the test data set
***
<div class="alert alert-block alert-info">
Reading data and getting parameters array
</div>

The above steps are implemented over the test data set.

In [ ]:
# let's switch to test directory

X_test = []

for dirname, _, filenames in os.walk('/kaggle/input/physiological-signals-processing-challenge-2021/data_challenge/test/'):
    filenames.sort()
    j = 0
    for filename in filenames:
        #print(filename)
        
        if j%25==0:
            clear_output(wait = True)
            print(j," of ",len(filenames)/2)
            #print(filename)
            
        if 'mat' in filename:
            continue
        #
        j+=1
        #read the pat data
        record = loadmat(os.path.join(dirname,filename[:-4]+'.mat'))
        
        #get the header
        with open(os.path.join(dirname,filename)) as f:
            header = f.readlines()

        
        mean_hr,std_hr,activity,mobility,complexity,entropy,en_a,en_p,en_r= get_parameters(record,header,plot_flag = False)
        
        X_test.append([int(filename[:-4]),mean_hr,std_hr,activity,mobility,complexity,entropy,en_a,en_p,en_r])

In [ ]:
# Handle problematic values:

X_test = np.array(X_test)
Z_test = clean_array(X_test)

# sort data

idx_sort = np.argsort(Z_test[:,0])

Z_test = Z_test[idx_sort,:]

<div class="alert alert-block alert-info">
Predicting and obtaining our final solution
</div>

Since our parameters were previously defined and assessed and our decisor was previously trained, we can jump to predict alarms of the data set recordings.

**Our solution is created!**

In [ ]:
#predict

y_hat_test = nb_detector.predict(Z_test[:,1:])

In [ ]:
print(y_hat_test[:])

#create solution

df = pd.DataFrame({'Id': Z_test[:,0], 'Category': y_hat_test})
df= df.astype(int)
#df = df.astype({"Id": int, "Category": float})

df.to_csv('submission_naive_bayes_final.csv',index = False)

In [ ]:
df.head(10)

***
### References
***

- Several StackOverflow issues
- Numpy documentation
- Scipy documentation
- Biosppy documentation
- [Hjorth descriptors alternative formulas](http://https://ruidera.uclm.es/xmlui/bitstream/handle/10578/15441/TFG-Luis%20Caba%c3%b1ero%20G%c3%b3mez.pdf?sequence=1&isAllowed=y)
- [Feature extraction Kaggle notebook](http://https://www.kaggle.com/treina/feature-extractor-matlab2python-translated)